In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer ##how we handle missing values
from sklearn.compose import ColumnTransformer ##Transform our data features
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt


In [ ]:
#SD DUI Crash Data
SD_DUIs_df = pd.read_csv("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/SD DUI Filtered.csv")

##LA DUI Crash Data
LA_DUIs_df = pd.read_csv("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/LA DUI Filtered.csv")

# Concatenate the two colision Dataframes vertically
all_dui_df = pd.concat([SD_DUIs_df, LA_DUIs_df])

# Reset the index of the combined DataFrame
all_dui_df.reset_index(drop=True, inplace=True)

# Assuming 'Latitude' and 'Longitude' columns exist in your DataFrame
geometry = [Point(xy) for xy in zip(all_dui_df['LONGITUDE'], all_dui_df['LATITUDE'])]
DUI_gdf = gpd.GeoDataFrame(all_dui_df, geometry=geometry)
DUI_gdf.set_crs("EPSG:4326")

##load census data
SD_Census = pd.read_csv("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/SD data.csv")

LA_Census = pd.read_csv("/mnt/c/Users/tbrag/OneDrive/Desktop/bda602venv/LA data.csv")

# Concatenate the census Dataframes vertically
all_census_df = pd.concat([SD_Census, LA_Census])
all_census_df.head(5)

#all_census_df['geometry'] = all_census_df['geometry'].apply(wkt.loads)
#Census_gdf = gpd.GeoDataFrame(all_census_df, geometry='geometry')
#Census_gdf.set_crs("EPSG:4326")

#DUI_gdf.crs = "EPSG:4326"
#Census_gdf.crs = "EPSG:4326"


In [ ]:
def count_points_in_polygons(points, polygons, polygon_id, new_column="points_count"):

    # Save the index to restore it later
    original_index = polygons.index

    # Ensures polygon_id is not the index but a column
    if original_index.name == polygon_id:
        polygons = polygons.reset_index()

    # Count points in polygons
    points_in_polygon = (
        # Spatial join associates points and polygons that intersects each other
        polygons.sjoin(
            points,
            how="inner",  # Only keep points that matches a polygon
            op='intersects',
        )
        .groupby(polygon_id)  # Group points by polygons
        .size()  # Get number of points
        .rename(new_column)  # Name your column as you want it to appear in polygons
    )

    # Add count series to the polygons dataframe
    polygons = (
        polygons.set_index(polygon_id)  # Ensures the index is the same as points_in_polygons
        .join(
            points_in_polygon,
            how="left",  # Keep polygons containing no points
        )
        .fillna({new_column: 0})  # Fill NaN with 0
    )

    if original_index.name != polygon_id:
        # Avoids duplicating polygon_id as column and index
        polygons = polygons.reset_index()

    polygons = polygons.set_index(original_index) # Restore the original index

    return polygons
new_df = count_points_in_polygons(DUI_gdf, Census_gdf, 'tract', new_column="points_count")

# Count the number of rows
num_rows = len(new_df)
print(f"Number of rows: {num_rows}")

# Sum the dui_count column
dui_count_sum = new_df['points_count'].sum()
print(f"Sum of DUI counts: {dui_count_sum}")

new_df.head()


In [ ]:
columns_to_keep = ['AIRLINE','ORIGIN','DEST','DISTANCE','ori_TMIN','ori_TMAX','ori_SNOW','ori_SNWD','ori_AWND','ori_PRCP','dest_TMIN','dest_TMAX','dest_SNOW','dest_SNWD','dest_AWND','dest_PRCP','15_DELAYED','year','month','day','hour','day_of_week']

# Drop all columns except the specified ones
df= new_df.drop(columns=new_df.columns.difference(columns_to_keep))

df.info()

In [ ]:
#extract Numeric Features
numeric_features = fw_data_final.select_dtypes(
    include=["float","int"]
)

categorical_features = fw_data_final.iloc[:,~fw_data_final.columns.isin(numeric_features.columns.values)]

